In [1]:
import calendar
import csv
import datetime
import itertools
import os
import random
from datetime import datetime, timedelta
from itertools import cycle, islice

import numpy as np
import pandas as pd

random.seed(42)

In [2]:
raw = pd.read_csv ('data/pizza_sales.csv')
raw["order_date"] = pd.to_datetime(raw["order_date"], format='%m-%d-%y')
raw.head()

,order_details_id,order_id,pizza_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name,first_name,second_name,address,date_of_birth
0,1,1,hawaiian_m,1,2015-01-01,11:38,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza,Darren,Lambert,"360 Holland Pine Suite 597\r\nMartinezview, SC...",1943-04-24
1,2,2,classic_dlx_m,1,2015-01-01,11:57,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza,Curtis,Clayton,"96436 Lisa Field\r\nEast John, DE 91152",1997-08-23
2,3,2,five_cheese_l,1,2015-01-01,11:57,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza,Curtis,Clayton,"96436 Lisa Field\r\nEast John, DE 91152",1997-08-23
3,4,2,ital_supr_l,1,2015-01-01,11:57,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza,Curtis,Clayton,"96436 Lisa Field\r\nEast John, DE 91152",1997-08-23
4,5,2,mexicana_m,1,2015-01-01,11:57,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza,Curtis,Clayton,"96436 Lisa Field\r\nEast John, DE 91152",1997-08-23


#### Generate random rows with outlying dates.


In [3]:
def generate_random_date_in_range(start_year, end_year):
    """Generate a random date within a specified range of years."""
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    return start_date + timedelta(days=random_days)


def generate_random_past_or_present_row(df):
    """Generate a new row with random values based on existing data, with a modified date.
    Dates are adjusted to fall within the ranges 1700-1800 or 2100-2200."""
    sample_row = df.sample(1, random_state=random.randint(1, 10000)).iloc[0].copy()

    # Choose between the past and future date ranges
    if random.choice([True, False]):
        start_year, end_year = 1700, 1800
    else:
        start_year, end_year = 2100, 2200

    # Generate a random date within the chosen range and keep it as a datetime object
    new_date = generate_random_date_in_range(start_year, end_year)

    # Update the order_date in the sample row
    sample_row["order_date"] = new_date

    # Ensure the order_details_id is unique
    sample_row["order_details_id"] = df["order_details_id"].max() + 1

    return sample_row


# Since the df was modified in previous operations, let's reset it to ensure consistency
df = raw.copy()

# Convert 'order_date' to datetime for consistency with the function's expectations

# Generate 60 new rows adjusted for the new date ranges
new_rows = [generate_random_past_or_present_row(df) for _ in range(60)]

# Convert the list of dictionaries to a DataFrame
df_new_rows = pd.DataFrame(new_rows)

# Add these rows to the original DataFrame
output = pd.concat([df, df_new_rows], ignore_index=True)

output.sort_values(by='order_date')

,order_details_id,order_id,pizza_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name,first_name,second_name,address,date_of_birth
48678,48621,14246,peppr_salami_m,1,1701-04-17,15:00,16.50,16.50,M,Supreme,"Genoa Salami, Capocollo, Pepperoni, Tomatoes, ...",The Pepper Salami Pizza,David,Lee,"0471 Hall Hill Suite 297\r\nSimpsonburgh, IA 1...",1965-01-09
48653,48621,14122,mediterraneo_m,1,1708-06-15,11:45,16.00,16.00,M,Veggie,"Spinach, Artichokes, Kalamata Olives, Sun-drie...",The Mediterranean Pizza,Nathan,Haynes,"2930 Kyle Radial Apt. 820\r\nNew Katelyn, MA 2...",1956-05-06
48676,48621,2319,prsc_argla_s,1,1711-06-29,15:10,12.50,12.50,S,Supreme,"Prosciutto di San Daniele, Arugula, Mozzarella...",The Prosciutto and Arugula Pizza,Mercedes,Hamilton,"1439 Fisher Summit Apt. 434\r\nPort Marco, VA ...",1975-11-01
48635,48621,9471,big_meat_s,1,1712-06-25,16:09,12.00,12.00,S,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sau...",The Big Meat Pizza,Susan,Porter,"8049 Kimberly Station\r\nJonathanberg, WY 30407",1985-06-24
48668,48621,5348,four_cheese_l,1,1714-02-18,17:37,17.95,17.95,L,Veggie,"Ricotta Cheese, Gorgonzola Piccante Cheese, Mo...",The Four Cheese Pizza,Kyle,Lang,"6741 Williams Road\r\nJessicaview, TN 77865",1940-08-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48638,48621,9558,prsc_argla_s,1,2181-05-10,19:27,12.50,12.50,S,Supreme,"Prosciutto di San Daniele, Arugula, Mozzarella...",The Prosciutto and Arugula Pizza,Tracy,Lindsey,"8454 Julian Glen Apt. 909\r\nPort John, FL 95707",1999-08-29
48648,48621,551,thai_ckn_l,1,2182-05-02,19:26,20.75,20.75,L,Chicken,"Chicken, Pineapple, Tomatoes, Red Peppers, Tha...",The Thai Chicken Pizza,Jason,Grant,USS Walter\r\nFPO AA 85393,1960-11-14
48662,48621,15540,spinach_supr_m,1,2191-02-03,12:52,16.50,16.50,M,Supreme,"Spinach, Red Onions, Pepperoni, Tomatoes, Arti...",The Spinach Supreme Pizza,Andrew,Hendrix,16457 Alison Terrace Suite 617\r\nSouth Leviha...,1968-08-26
48656,48621,5401,soppressata_m,1,2194-12-10,15:42,16.50,16.50,M,Supreme,"Soppressata Salami, Fontina Cheese, Mozzarella...",The Soppressata Pizza,Michael,Foster,147 Williams Ridges Suite 382\r\nEast Elizabet...,1957-04-01


### Adding random, duplicate rows.

In [4]:
df = raw.copy()

random_sample = df.sample(n=1000, random_state=42)
output = output.append(random_sample, ignore_index=True)
output

C:\Users\Chimney\AppData\Local\Temp\ipykernel_13980\1031210640.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(random_sample, ignore_index=True)


,order_details_id,order_id,pizza_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name,first_name,second_name,address,date_of_birth
0,1,1,hawaiian_m,1,2015-01-01,11:38,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza,Darren,Lambert,"360 Holland Pine Suite 597\r\nMartinezview, SC...",1943-04-24
1,2,2,classic_dlx_m,1,2015-01-01,11:57,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza,Curtis,Clayton,"96436 Lisa Field\r\nEast John, DE 91152",1997-08-23
2,3,2,five_cheese_l,1,2015-01-01,11:57,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza,Curtis,Clayton,"96436 Lisa Field\r\nEast John, DE 91152",1997-08-23
3,4,2,ital_supr_l,1,2015-01-01,11:57,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza,Curtis,Clayton,"96436 Lisa Field\r\nEast John, DE 91152",1997-08-23
4,5,2,mexicana_m,1,2015-01-01,11:57,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza,Curtis,Clayton,"96436 Lisa Field\r\nEast John, DE 91152",1997-08-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49675,13997,6136,classic_dlx_s,1,2015-04-13,18:13,12.00,12.00,S,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza,Jessica,Lee,"7828 Lindsey Bypass\r\nMurilloton, SC 85344",1977-11-19
49676,15227,6679,pepperoni_m,1,2015-04-22,18:50,12.50,12.50,M,Classic,"Mozzarella Cheese, Pepperoni",The Pepperoni Pizza,Peter,Hunter,"87765 Paul Avenue\r\nJohnsonshire, CO 64161",1951-07-25
49677,28225,12436,thai_ckn_l,1,2015-07-27,11:55,20.75,20.75,L,Chicken,"Chicken, Pineapple, Tomatoes, Red Peppers, Tha...",The Thai Chicken Pizza,Aaron,Thompson,"796 James Shore\r\nLongland, AZ 47752",1961-04-22
49678,36793,16238,ital_supr_m,1,2015-10-01,12:19,16.50,16.50,M,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza,Tyler,Smith,"114 Justin Squares Suite 096\r\nEdwardview, OR...",1987-05-08


#### Change random values in random rows to null.

In [5]:
def modify_random_rows(df, random_seed=42):
    np.random.seed(random_seed)  # Set the random seed for reproducibility

    # Exclude columns of datetime64[ns] type from modification
    columns_to_consider = df.select_dtypes(exclude=['datetime64[ns]']).columns
    
    # Determine the exact number of rows to modify, randomly choosing up to 1000
    rows_to_modify_count = np.random.randint(1, min(len(df), 1001))
    
    # Select random row indices
    random_indices = np.random.choice(df.index, size=rows_to_modify_count, replace=False)
    
    for index in random_indices:
        # Randomly determine the number of values to change to null, up to 5
        values_to_null_count = np.random.randint(1, 6)
        
        # Select random columns to modify from the eligible columns
        if len(columns_to_consider) > 0:
            columns_to_modify = np.random.choice(columns_to_consider, size=min(values_to_null_count, len(columns_to_consider)), replace=False)
            for column in columns_to_modify:
                df.at[index, column] = np.nan
    
    return df

# To use this function, simply pass your dataframe to it as follows:
# Display the modified rows
output = modify_random_rows(output)
output.loc[output.isnull().any(axis=1)]

,order_details_id,order_id,pizza_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name,first_name,second_name,address,date_of_birth
147,148.0,NaN,spinach_supr_m,1.0,2015-01-01,20:50,16.50,NaN,M,Supreme,NaN,The Spinach Supreme Pizza,Travis,Dixon,NaN,1989-05-05
199,200.0,85.0,NaN,1.0,2015-01-02,NaN,NaN,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza,David,Miller,"312 Tyler Mills Suite 909\r\nBenitezshire, TN ...",1940-02-26
292,NaN,123.0,NaN,NaN,2015-01-02,20:12,12.00,12.00,NaN,Classic,"Capocollo, Red Peppers, Tomatoes, Goat Cheese,...",The Italian Capocollo Pizza,Monica,Thomas,"4229 Williams Field Suite 698\r\nWest Toni, HI...",1982-08-25
1219,1220.0,NaN,NaN,1.0,2015-01-09,17:23,12.00,NaN,NaN,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza,Raven,Bates,"75677 Woods Village\r\nLake Shelly, AZ 33231",1954-01-06
1388,1389.0,618.0,spin_pesto_m,1.0,2015-01-10,20:31,16.50,NaN,M,Veggie,"Spinach, Artichokes, Tomatoes, Sun-dried Tomat...",The Spinach Pesto Pizza,NaN,Faulkner,NaN,1963-01-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47409,NaN,NaN,southw_ckn_m,1.0,2015-12-20,13:21,NaN,16.75,M,Chicken,"Chicken, Tomatoes, Red Peppers, Red Onions, Ja...",The Southwest Chicken Pizza,Tiffany,NaN,NaN,1990-05-12
48489,48490.0,21295.0,mexicana_m,1.0,2015-12-31,14:52,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza,Vicki,NaN,"762 Sabrina Tunnel\r\nNew Jessicashire, AZ 14063",1953-10-22
48619,48620.0,21350.0,bbq_ckn_s,1.0,2015-12-31,23:02,12.75,12.75,S,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers,...",The Barbecue Chicken Pizza,NaN,NaN,NaN,NaN
49060,18537.0,8138.0,napolitana_s,1.0,2015-05-16,17:30,12.00,12.00,S,Classic,"Tomatoes, Anchovies, Green Olives, Red Onions,...",The Napolitana Pizza,NaN,Howard,"108 Summers Meadow\r\nEast Mark, DE 12468",1981-08-28


#### Swap values between columns.

In [6]:
def swap_values_between_columns(df, random_seed=42):
    np.random.seed(random_seed)  # Set the random seed for reproducibility
    
    # Determine the exact number of values to swap, up to 1000 or the length of the dataframe, whichever is smaller
    values_to_swap_count = np.random.randint(1, min(len(df), 1001))
    
    # Select random indices to swap values at
    random_indices = np.random.choice(df.index, size=values_to_swap_count, replace=False)
    
    # Perform the swap between 'pizza_name' and 'order_id' for the selected indices
    for index in random_indices:
        df.at[index, 'pizza_name'], df.at[index, 'order_id'] = df.at[index, 'order_id'], df.at[index, 'pizza_name']
    
    return df

# Usage example:
output = swap_values_between_columns(output)
# This will randomly swap values between 'pizza_name' and 'order_id' in the dataframe.

In [7]:
output.to_csv(
    "data/pizza_sales_with_data_quality_issues.csv",
    index=False,
)

In [8]:
output

,order_details_id,order_id,pizza_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name,first_name,second_name,address,date_of_birth
0,1.0,1.0,hawaiian_m,1.0,2015-01-01,11:38,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza,Darren,Lambert,"360 Holland Pine Suite 597\r\nMartinezview, SC...",1943-04-24
1,2.0,2.0,classic_dlx_m,1.0,2015-01-01,11:57,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza,Curtis,Clayton,"96436 Lisa Field\r\nEast John, DE 91152",1997-08-23
2,3.0,2.0,five_cheese_l,1.0,2015-01-01,11:57,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza,Curtis,Clayton,"96436 Lisa Field\r\nEast John, DE 91152",1997-08-23
3,4.0,2.0,ital_supr_l,1.0,2015-01-01,11:57,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza,Curtis,Clayton,"96436 Lisa Field\r\nEast John, DE 91152",1997-08-23
4,5.0,2.0,mexicana_m,1.0,2015-01-01,11:57,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza,Curtis,Clayton,"96436 Lisa Field\r\nEast John, DE 91152",1997-08-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49675,13997.0,6136.0,classic_dlx_s,1.0,2015-04-13,18:13,12.00,12.00,S,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza,Jessica,Lee,"7828 Lindsey Bypass\r\nMurilloton, SC 85344",1977-11-19
49676,15227.0,6679.0,pepperoni_m,1.0,2015-04-22,18:50,12.50,12.50,M,Classic,"Mozzarella Cheese, Pepperoni",The Pepperoni Pizza,Peter,Hunter,"87765 Paul Avenue\r\nJohnsonshire, CO 64161",1951-07-25
49677,28225.0,12436.0,thai_ckn_l,1.0,2015-07-27,11:55,20.75,20.75,L,Chicken,"Chicken, Pineapple, Tomatoes, Red Peppers, Tha...",The Thai Chicken Pizza,Aaron,Thompson,"796 James Shore\r\nLongland, AZ 47752",1961-04-22
49678,36793.0,16238.0,ital_supr_m,1.0,2015-10-01,12:19,16.50,16.50,M,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza,Tyler,Smith,"114 Justin Squares Suite 096\r\nEdwardview, OR...",1987-05-08
